In [ ]:
#Import packages
import sys
sys.path.insert(0, '') ## Insert in the quotes the path to where QuantSIM folder is in your computer.

import numpy as np
import pandas as pd
import re
import os
import copy
import matplotlib.pyplot as plt
from pathlib import Path
import smbclient
import time
import shutil
import itertools
import bisect
from czifile import imread, CziFile
import napari
import scipy.ndimage as scipy_image
import pyclesperanto_prototype as cle
import scipy
import seaborn as sns
import xml.etree.ElementTree as ET
import warnings
from tqdm import tqdm
import traceback
import gzip

from QuantSIM.common import check_to_make_dir
from QuantSIM.SIM_segment import SIM_segmentation, slide_channel_threshold
from QuantSIM.SIM_analyze import SIM_analysis

In [ ]:
##SIM_BLL_RIF1

# Identification of nuclei by manually drawing masks before running pipeline.
# 3 channels: EdU (clicked, AF647), Cy3-dUTP, SON speckles (immunostained, AF488).

## Insert in the quotes the path to where you store 3D-SIM stack files (czi format) are in your computer.
# The expected paths are:
# 3D-SIM stack files: MAIN_PATH/RAWS_FOLDER/EXP_FOLDER/COND_FOLDER/ _Out.czi files
# Mask files: MAIN_PATH/ROI_FOLDER/EXP_FOLDER/COND_FOLDER/ _dapi.czi files
# output files: MAIN_PATH/RES_FOLDER/EXP_FOLDER_AUTO_ANALYSIS/slide/ many files organized in layers (viewable with Napari), tables, overlap_layers, overlap_tables

MAIN_PATH = ''
RAWS_FOLDER = 'raw_files'
ROI_FOLDER = 'raw_masks'
EXP_FOLDER = 'SIM_BLL_RIF1'
RES_FOLDER = 'segmented/OTSU_THRESHOLDING'

EXP_PATH = os.path.join(MAIN_PATH, RAWS_FOLDER, EXP_FOLDER)
ROI_PATH = os.path.join(MAIN_PATH, ROI_FOLDER, EXP_FOLDER)
RES_PATH = os.path.join(MAIN_PATH, RES_FOLDER, EXP_FOLDER + '_AUTO_ANALYSIS')
#RES_MAIN_FOLDER = EXP_PATH[::-1].split('/', 1)[0][::-1] + '_AUTO_ANALYSIS' ## Double reverse search: Reverse to pick last element after splitting (which is first) and reverse again to get original substring.
cle.select_device('M1')
if not os.path.exists(EXP_PATH):
    raise Exception('The experimental data path indicated could not be found.')

if not os.path.exists(ROI_PATH):
    raise Exception('Nuclei masks path indicated could not be found.')

if not os.path.exists(RES_PATH):
    check_to_make_dir(RES_PATH, known_path = MAIN_PATH)

EXISTING_PATH = RES_PATH
threshold_method = 'otsu'
spot_sigma = 1
outline_sigma = 0
min_size = 50 # Considering a pixel as a cube, volume approx. 16000 nm**3. Nucleosome volume is 665 nm**3, 24 fully-packed nucleosomes/pixel volume (but chromatin status, signal intensity and light diffraction may alter nucleosome relative size).
seg_keywords = ['raw','trim']
min_size_overlap = 0
pixel_to_nano_conversion = 25
unit = 'nano'

foldernames = ['raw','trim']

for foldername,seg_keyword in zip(foldernames,seg_keywords):
    RES_FOLDERNAME =  foldername+'__ss_{}__os_{}__min_size_{}__min_size_overlap_{}'.format(str(spot_sigma),str(outline_sigma), str(min_size), str(min_size_overlap))
    RES_FOLDERNAME = os.path.join(RES_PATH, RES_FOLDERNAME)
    for slide in os.listdir(EXP_PATH):
        SLIDE_PATH = os.path.join(EXP_PATH, slide)
        SLIDE_MASKS = os.path.join(ROI_PATH, slide)
        if os.path.isdir(SLIDE_PATH) and os.path.isdir(SLIDE_MASKS):
            czi_files = np.array(os.listdir(SLIDE_PATH))
            czi_files = czi_files[['.czi' in czi_file for czi_file in czi_files]]
            
            # It is possible that there are multiple masks per image, or that there are multiple images with the same mask. The important thing is Image ID is the same to cross-check.
            mask_files = np.array(os.listdir(SLIDE_MASKS))
            mask_files = mask_files[['.tif' in mask_file for mask_file in mask_files]]
            mask_id = [[],[]]
            for mask_file in mask_files:
                file_match = re.match(r'Image (\d+).*\.tif$', mask_file)
                mask_id[0].append(file_match.group(0))
                mask_id[1].append(file_match.group(1))
            mask_id = np.array(mask_id)

            # Link every image with every mask that share ID
            linked_files_slide = []   ## Nested list. Lists inside have two elements: dapi filename and other channels filename
            for czi_file in czi_files:
                file_match = re.match(r'Image (\d+).*\.czi$', czi_file)
                id_matches = np.where(mask_id[1] == file_match.group(1))[0].tolist() #Returns a tuple, each element would be dimensions of array. Since it is 1D-array, extract first element.
                if len(id_matches) > 0:
                    for id_match_pos in id_matches:
                        linked_files_slide.append([czi_file,mask_id[0][id_match_pos]])
            
            # Now, for every pair identified, do analysis only if it is not the unprocessed image
            for linked_files_image in linked_files_slide:
                czi_file = linked_files_image[0]
                mask_file = linked_files_image[1]
                czi_match = re.match(r'Image \d+(.*)\.czi$', czi_file)
                mask_match = re.match(r'Image \d+(.*)\.tif$', mask_file)
                if czi_match != None:
                    if czi_match.group(1) != '':
                        RES_IMAGE_PATH = os.path.join(RES_FOLDERNAME, slide, czi_file[:-4] + '_' + mask_match.group(1)) # -4 because we remove .czi for naming the folder. Added mask_match to not overwrite images with multiple masks.
                        if os.path.isdir(RES_IMAGE_PATH):
                            continue
                        check_to_make_dir(RES_IMAGE_PATH)
                        EXISTING_PATH = RES_IMAGE_PATH
                        MASK_PATH = os.path.join(SLIDE_MASKS, mask_file)
                        IMAGE_PATH = os.path.join(SLIDE_PATH, czi_file)
                    else:
                        continue
                else:
                    continue
                channel_names = ['EdU','dUTP']
                channel_colors = ['magenta','yellow']
                dist_calculation_1 = dict(zip(channel_names,['centroid','centroid']))
                dist_calculation_2 = dict(zip(channel_names,['maxima','maxima']))
                channel_laser_wavelengths = []
                metadata = CziFile(IMAGE_PATH)
                xml_metadata = ET.ElementTree(ET.fromstring(metadata.metadata()))
                root = xml_metadata.getroot()
                for element in xml_metadata.iter('ExcitationWavelength'):
                    channel_laser_wavelengths.append(element.text)
                channel_laser_wavelengths = channel_laser_wavelengths[:int(round(len(channel_laser_wavelengths)/2,0))]
                channel_laser_wavelengths = [int(round(float(i),0)) for i in channel_laser_wavelengths]
                wavelengths_np = np.array(channel_laser_wavelengths)
                channel_org = list(range(len(channel_laser_wavelengths)))
                channel_org = ['skip_channel' for i in channel_org]
                channel_org[np.where(wavelengths_np == 642)[0][0]] = 'EdU'
                channel_org[np.where(wavelengths_np == 561)[0][0]] = 'dUTP'
                Exp = SIM_segmentation(IMAGE_PATH, RES_IMAGE_PATH,
                                    channel_names = channel_names,
                                    channel_colors = channel_colors,
                                    channel_org = channel_org,
                                    MASK_PATH=MASK_PATH,
                                    override = True)
                Exp.anisotropy_correction()
                Exp.image_truncation()
                Exp.make_spatial_mask(prism_mode = True)
                ## We are reusing the global threshold from whole image to do segmentation. The other option is redo the threshold in the cropped image, which may have slight changes (or a lot if the cell was dim in a channel).
                Exp.segmentation_channel_intensity_based('EdU', spot_sigma = spot_sigma, outline_sigma = outline_sigma,
                                                        inherit_threshold = Exp.channels_threshold_trunc['EdU'], use_roi_mask = True)
                Exp.segmentation_channel_intensity_based('dUTP', spot_sigma = spot_sigma, outline_sigma = outline_sigma,
                                                        inherit_threshold = Exp.channels_threshold_trunc['dUTP'], use_roi_mask = True)
                Exp.select_segmentation(seg_keyword)
                Exp.save_segmentation_layers(prefix='ALL_REGIONS__')
                Exp.remove_small_regions('EdU',seg_keyword, min_size)
                Exp.remove_small_regions('dUTP',seg_keyword, min_size)
                ## Check if Exp has regions. If empty, skip to next image.
                Exp.select_segmentation(seg_keyword)
                if any(len(Exp.tables['004_segmented_regions'][channel].index) < 2 for channel in channel_names):
                    continue
                Exp.dist_to_core_and_rim()
                Exp.save_segmentation_layers()
                with open(os.path.join(RES_IMAGE_PATH,'000 SIM_segmentation extra info.txt'),'w') as txtfile:
                    txtfile.write(str(dict(zip(channel_names,list(zip(channel_colors,channel_laser_wavelengths))))))
                    txtfile.write('\nScaling: {}'.format(str(Exp.voxelsizes)))
                    txtfile.write('\nInitial threshold method to find overall signal regions: {}'.format(threshold_method))
                    txtfile.write('\nThreshold calculated from max-z projection for truncation:')
                    txtfile.write('\n'+str(Exp.channels_threshold_trunc))
                    txtfile.write('\nCoordinates of truncation corner reference (YX):')
                    txtfile.write('\n'+str(Exp.trunc_refcoord))
                    txtfile.write('\nThreshold calculated from max-z projection for segmentation:')
                    txtfile.write('\n'+str(Exp.channels_threshold_seg))
                    txtfile.write('\nSpot sigma (local maxima identification)')
                    txtfile.write('\n'+str(spot_sigma))
                    txtfile.write('\nOutline sigma (Voronoi-watershed)')
                    txtfile.write('\n'+str(outline_sigma))
                    txtfile.write('\nMinimum segmented region size (pixels)')
                    txtfile.write('\n'+str(min_size))
                    txtfile.write('\nSegmentation selected')
                    txtfile.write('\n'+str(seg_keyword))
                    txtfile.write('\nMinimum overlap size (pixels)')
                    txtfile.write('\n'+str(min_size_overlap))
                    txtfile.write('\nunit conversion factor: ')
                    txtfile.write('\n'+str(pixel_to_nano_conversion))
                    txtfile.write('\nunit converted to: ')
                    txtfile.write('\n'+str(unit))
                for channel in channel_names:
                    Exp_channel = SIM_analysis(Exp, channel_1 = channel, channel_2 = channel)
                    Exp_channel.neighbor_distance()
                    Exp_channel.neighbor_distance(channels_ref = ['maxima','maxima'])
                    Exp_channel.overlap_detection()
                    Exp_channel.overlap_declumping()
                    Exp_channel.two_channel_overlap_regions(overlap_stoich=(1,1))
                    Exp_channel.measure_overlapping_regions(pixel_unit_conversion = 1, unit = 'pixel')
                    Exp_channel.measure_overlapping_regions(pixel_unit_conversion = pixel_to_nano_conversion, unit = unit)
                    Exp_channel.single_channel_cleanup()
                    Exp_channel.save_segmentation_layers(prefix = 'one_channel__')
                    del Exp_channel
                for combo in itertools.combinations(channel_names, 2):
                    Exp_overlap = SIM_analysis(Exp, channel_1 = combo[0], channel_2 = combo[1])
                    Exp_overlap.neighbor_distance(channels_ref = [dist_calculation_1[combo[0]],dist_calculation_1[combo[1]]])
                    Exp_overlap.neighbor_distance(channels_ref = [dist_calculation_2[combo[0]],dist_calculation_2[combo[1]]])
                    Exp_overlap.overlap_detection(min_size_overlap = min_size_overlap)
                    if all(['EdU' in combo,'dUTP' in combo]):
                        Exp_overlap.overlap_declumping()
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_common__')
                    ## PROBLEM: keep calculating things when they have no data, no overlaps to work with, should be empty but then gets confused...
                    ## SOLUTION: ignore instances where there are less than 2 intersecting regions identified.
                    if len(Exp_overlap.tables['102_intersection_regions'].index) < 2:
                        with open(os.path.join(RES_IMAGE_PATH,'000 SIM_analysis {}_vs_{} extra info.txt'.format(str(combo[0]),str(combo[1]))), 'w') as txtfile:
                            txtfile.write('Channels evaluated for overlap: {}'.format(str(combo)))
                            for i in range(len(combo)):
                                txtfile.write('\n'+str(combo[i]))
                                txtfile.write('\n\tValue\tCount')
                                txtfile.write('\n'+str(Exp_overlap.tables['102_overlap_bool'].sum(axis = 1-i).value_counts().sort_index()))
                                txtfile.write('\nTotal: '+ str(Exp.tables['004_segmented_maxima'][combo[i]].shape[0]))
                            txtfile.write('\nFork structure number')
                            txtfile.write('\nInitiation: NA')
                            txtfile.write('\nOngoing: NA')
                            txtfile.write('\nTermination: NA')
                        del Exp_overlap
                        continue
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(0,0))
                    Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = 1, unit = 'pixel')
                    #Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = pixel_to_nano_conversion, unit = unit)
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_no_overlap__', common_results = False)
                    del Exp_overlap.layers['105_stoich_0-to-0_segmented_labeled']
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(1,1))
                    Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = 1, unit = 'pixel')
                    #Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = pixel_to_nano_conversion, unit = unit)
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_ongoing__', common_results = False)
                    ong_n = Exp_overlap.tables['105_overlap_labels'].shape[0]
                    del Exp_overlap.layers['105_stoich_1-to-1_segmented_labeled']
                    del Exp_overlap.tables['106_channel_measurements']
                    del Exp_overlap.tables['106_overlap_measurements']
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(2,1))
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_initiation__', common_results = False)
                    init_n = Exp_overlap.tables['105_overlap_labels'].shape[0]
                    del Exp_overlap.layers['105_stoich_2-to-1_segmented_labeled']
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(1,2))
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_termination__', common_results = False)
                    term_n = Exp_overlap.tables['105_overlap_labels'].shape[0]
                    with open(os.path.join(RES_IMAGE_PATH,'000 SIM_analysis {}_vs_{} extra info.txt'.format(str(combo[0]),str(combo[1]))), 'w') as txtfile:
                        txtfile.write('Channels evaluated for overlap: {}'.format(str(combo)))
                        for i in range(len(combo)):
                            txtfile.write('\n'+str(combo[i]))
                            txtfile.write('\n\tValue\tCount')
                            txtfile.write('\n'+str(Exp_overlap.tables['102_overlap_bool'].sum(axis = 1-i).value_counts().sort_index()))
                            txtfile.write('\nTotal: '+ str(Exp.tables['004_segmented_maxima'][combo[i]].shape[0]))
                        txtfile.write('\nFork structure number')
                        txtfile.write('\nInitiation: '+ str(init_n))
                        txtfile.write('\nOngoing: '+ str(ong_n))
                        txtfile.write('\nTermination: '+ str(term_n))
                    del Exp_overlap
                del Exp